# Assignment #4 - Reinforcement Learning

<!-- ### <font color="red"> DUE: Nov 26 (Tuesday) 11:00 pm </font>   -->

<font color="blue"> jawad chowdhury </font>

# I. Overview

<!-- Describe the objective of this assignment. You can briefly state how you accompilsh it. -->

The main objective of this assignment was to implement a modified version of Game Rummy by training an agent with Re-inforcement learning.

Here, i get familiarized with the basic Reinforcement environment including states, actions, rewards etc.

And also got some idea about different approaches such as SAARSA, Q-learning and some sub approaches like epsilon-greedy.

<!-- # II. Problems  -->
<!-- ## Rummy (Our Way) -->
<!-- ![](https://upload.wikimedia.org/wikipedia/commons/9/9a/3_playing_cards.jpg) -->
<!-- ### Description: -->
<!-- Rummy is mostly played by 4 players where each player get 10 cards and rest of placed as closed deck.  

But for this assignment to keep it simple, we have changed some rules, reduced the deck of cards and distribution of the cards among each player. Otherwise, the our Q table would be large to handle on our local machines.  

So in this game each player will get 3 cards randomly from 'A' to '5'. With each turn you have to pick a card either from the pile of closed or open cards depending on the cards you have. After that you have to select a card to drop to the pile from your deck. Your goal is to make all of your 3 cards of same number. As soon as one player reaches the goal state, it will meld the cards and game will stop and all other player will get score as the sum of their cards in hand. Player with minimum score will win the game.  

Each player will play for the 10 rounds and if nobody reach to the meld stage then each will player will get score equivalent to the sum of their cards in hand.

Player with the lowest score wins the game.
 -->


<!-- ### <font color="red"> Game Introduction and Rules: </font> -->
<!-- 
You can only till max turns which is now 20. If you are able to meld your cards, you win otherwise you lose points as per your stash score.

Lets Simulate a Game Round :
The Goal of this game is to collect cards with the same rank.
Naming Conventions to follow H : Hearts, S : Spades, D : Diamonds, C : Clubs. Stash/Hand are the cards which you have. Stash/Hand Score: The sum of the rank of cards. 

- On the first turn you will be given 3 cards from the randomly shuffled deck. Eg: 3H, 2S, AD. Stash Score: 6
- In the next step you will have to either pick a card from pile or the deck. On the first turn, the top card of the deck is on the pile. The card on the pile is visible to you. So Card on the Pile : 2D . You have to decide whether you have to pick this card or you have to pick from pile. Since you alread
- Following the pickup step, you will have to drop one card from the stash/hand. Your Hand : 3H, 2S, AD, 2D
- You can drop any of these cards, but remember if you drop 2S or 2D you might not be able to make a series. So you have to decide whether to drop 3H or AD. Since there are maximum 20 turns, you have to keep your score low as well, Higher the score the higher points you lose. So you decide to drop 3H. Your hand/stash will be: 2S, 2D, AD
- Your opponent will play his hand, he/she shall pick up a card from a pile or deck and the drop a card to the pile.
- Now its your turn. The card on the Pile : 5H. Since picking up this card won't help you much. You decide to pick a card from the Deck. You get lucky, you get a 2C card. Your hand now is 2S, 2D, 2C, AD. Hurray! You can now meld these cards and drop the last card to make your score 0.
- You win this round. Your opponent Stash/Hand score : 15. So you are at 0 points and your opponent is at 15 points. You are ahead of him since you have to keep your score as low as possible.
 -->

## Rummy Environment

In [25]:
import random 
from functools import reduce
from collections import defaultdict
import numpy as np
from copy import copy
%matplotlib inline

Here, we define the SUITS, RANKS of the cards and their RANK_VALUE.

In [26]:
SUIT = ['H','S','D','C']
RANK = ['A', '2', '3', '4', '5','6','7']
RANK_VALUE = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'Q': 10, 'K': 10}

## Card Class Definition
<!-- __init__  : Defines the card details such as rank, suit and calculates the rank value -->


In [27]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.rank_to_val = RANK_VALUE[self.rank]

    def __str__(self):
        return f'{self.rank}{self.suit}'

    def __repr__(self):
        return f'{self.rank}{self.suit}'

    def __eq__(self, other):
        return self.rank == other.rank and self.suit == other.suit

## Deck Class Definition
<!-- __shuffle__ : Shuffles the deck in random order -->

<!-- __draw_card__ : Draws a card from the top of the deck -->

In [28]:

class Deck:
    def __init__(self, packs):
        self.packs = packs
        self.cards = []
        for pack in range(0, packs):
            for suit in SUIT:
                for rank in RANK:
                    self.cards.append(Card(rank, suit))

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        card = self.cards[0]
        self.cards.pop(0)
        return card


## Player Class:
<!-- 
### 1.__init__(self,name,stash=list(),isBot=False): 
Initializing stash, name, isBot/dealer points for each player.

### 2. deal_card(self,card):
This method appends the card in the stash and check the condition that length of stash should not be greater than nuber of cards length in game.

### 3. drop_card(self,card):
This method removes the card from stash and add that card into pile.

### 4. meld(self):
This method tries to find the cards with the same rank in the hand. If it finds then it will merge the cards in the hand to the melded cards array in the game. 

### 5. stash_score(self):
This method calculates sum of all the cards in stash according to the rank of each card.

### 6. get_info(self,debug):
This function fetch all the information of the player. -->

In [29]:

class Player:
    """
        Player class to create a player object.
        eg: player = Player("player1", list(), isBot = False)
        Above declaration will be for your agent.
        All the player names should be unique or else you will get error.

    """

    def __init__(self, name, stash=list(), isBot=False, points=0, conn=None):
        self.stash = stash
        self.name = name
        self.game = None
        self.isBot = isBot
        self.points = points
        self.conn = conn

    def deal_card(self, card):
        try:
            self.stash.append(card)
            if len(self.stash) > self.game.cardsLength + 1:
                raise ValueError('Cannot have cards greater than ')
        except ValueError as err:
            print(err.args)

    def drop_card(self, card):
        self.stash.remove(card)
        self.game.add_pile(card)
        return -1

    def meld(self):
        card_hash = defaultdict(list)
        for card in self.stash:
            card_hash[card.rank].append(card)
        melded_card_ranks = []
        for (card_rank, meld_cards) in card_hash.items():
            if len(meld_cards) >= 3:
                self.game.meld.append(meld_cards)
                melded_card_ranks.append(card_rank)
                for card in meld_cards:
                    self.stash.remove(card)

        for card_rank in melded_card_ranks:
            card_hash.pop(card_rank)
        return len(melded_card_ranks) > 0

    def stash_score(self):
        score = 0
        for card in self.stash:
            score += RANK_VALUE[card.rank]
        return score

    def get_info(self, debug):
        if debug:
            print(
                f'Player Name : {self.name} \n Stash Score: {self.stash_score()} \n Stash : {", ".join(str(x) for x in self.stash)}')
        card_ranks = []
        card_suits = []
        pileset = None
        pile = None
        for card in self.stash:
            card_ranks.append(RANK_VALUE[card.rank])
            card_suits.append(card.suit)
        if len(self.game.pile) > 0:
            return {
                "Stash Score": self.stash_score(),
                "CardSuit": card_suits,
                "CardRanks": card_ranks,
                "PileRank": self.game.pile[-1].rank,
                "PileSuit": self.game.pile[-1].suit
            }

        return {
            "Stash Score": self.stash_score(),
            "CardSuit": card_suits,
            "CardRanks": card_ranks
        }


<!-- ## Game Enviroment: -->


In [45]:

class RummyAgent():
    """
    Simple Rummy Environment

    Simple Rummy is a game where you need to make all the cards in your hand same before your opponent does.
    Here you are given 3 cards in your hand/stash to play.
    For the first move you have to pick a card from the deck or from the pile.
    The card in deck would be random but you can see the card from the pile.
    In the next move you will have to drop a card from your hand.
    Your goal is to collect all the cards of the same rank.
    Higher the rank of the card, the higher points you lose in the game.
    You need to keep the stash score low. Eg, if you can AH,7S,5D your strategy would be to either find the first pair of the card or by removing the highest card in the deck.
    You only have 20 turns to either win the same or collect low scoring card.
    You can't see other players cards or their stash scores.

    Parameters
    ====
    players: Player objects which will play the game.
    max_card_length : Number of cards each player can have
    max_turns: Number of turns in a rummy game
    """

    def __init__(self, players, max_card_length=5, max_turns=20):
        self.max_card_length = max_card_length
        self.max_turns = max_turns
        self.reset(players)

    def update_player_cards(self, players):
        for player in players:
            player = Player(player.name, list(), isBot=player.isBot, points=player.points, conn=player.conn)
            stash = []
            for i in range(self.max_card_length):
                player.stash.append(self.deck.draw_card())
            player.game = self
            self.players.append(player)
        self.pile = [self.deck.draw_card()]

    def add_pile(self, card):
        if len(self.deck.cards) == 0:
            self.deck.cards.extend(self.pile)
            self.deck.shuffle()
            self.pile = []
        self.pile.append(card)

    def pick_card(self, player, action):
        before_unique_rank_list = list(set([card.rank_to_val for card in player.stash]))
        before_unique_length = len(before_unique_rank_list)
        ss_before = int(player.stash_score())
        if action == 0:
            self.pick_from_pile(player)
        else:
            self.pick_from_deck(player)
        after_unique_rank_list = list(set([card.rank_to_val for card in player.stash]))
        after_unique_length = len(after_unique_rank_list)
        ss_after = int(player.stash_score())
        ss_delta = ss_after - ss_before

        s = [player.stash[0].rank_to_val, player.stash[1].rank_to_val, player.stash[2].rank_to_val, player.stash[3].rank_to_val]

        if player.meld():
            reward = 100
        elif after_unique_length == before_unique_length:
            reward = 90
        else:
            reward = -3 * ss_delta
        return {"reward": reward, "state": s}



    def pick_from_pile(self, player):
        card = self.pile[-1]
        self.pile.pop()
        return player.stash.append(card)

    def pick_from_deck(self, player):
        return player.stash.append(self.deck.draw_card())

    def get_player(self, player_name):
        return_player = [player for player in self.players if player.name == player_name]
        if len(return_player) != 1:
            print("Invalid Player")
            return None
        else:
            return return_player[0]

    def drop_card(self, player, card):
        before_unique_rank_list = list(set([card.rank_to_val for card in player.stash]))
        before_unique_length = len(before_unique_rank_list)
        ss_before = int(player.stash_score())
        player.drop_card(card)
        after_unique_rank_list = list(set([card.rank_to_val for card in player.stash]))
        after_unique_length = len(after_unique_rank_list)
        ss_after = int(player.stash_score())
        ss_delta = ss_after - ss_before

        if before_unique_length  == after_unique_length:
            reward = -90
        else:
            reward = -3 * ss_delta
        return {"reward": reward}

    def computer_play(self, player):
        # Gets a card from deck or pile
        if random.randint(0, 1) == 1:
            self.pick_from_pile(player)
        else:
            self.pick_from_deck(player)

        # tries to meld if it can
        #         if random.randint(0,10) > 5 :
        player.meld()

        # removes a card from the stash
        if len(player.stash) != 0:
            card = player.stash[(random.randint(0, len(player.stash) - 1))]
            player.drop_card(card)

    def play(self):
        for player in self.players:
            if len(player.stash) == 0:
                return True
        if self.max_turns <= 0:
            return True
        return False

    def _update_turn(self):
        self.max_turns -= 1

    def reset(self, players, max_turns=20):
        self.players = []
        self.deck = Deck(1)
        self.deck.shuffle()
        self.meld = []
        self.pile = []
        self.max_turns = max_turns
        self.update_player_cards(players)


<!-- ### Example Run with Random Players -->
<!-- This following code shows how to create an environment and players to play 10 games with 20 maximum rounds. This only includes random picking up of card and dropping.   -->

# III. Methods

- Decide your TD learning approach: SARSA or Q-learning? 
- Decide your function approximator.
- Describe your approach and the reason why you select it.
- Finish epsilon_greedy function and other TODOs. Explain it.
- Explain your codes.

## RLAgent for Rummy

<!-- Referencing the RLAgent in the lecture note, make your own agent class for the game.  -->

In [46]:
def coord_convert(s, sz):
    return [s[1], sz[0]-s[0]-1]

In [121]:

class RLAgent:
    """
        Reinforcement Learning Agent Model for training/testing
        with Tabular function approximation

    """

    def printQ(self):
        for i in range(7):
            for j in range(7):
                for k in range(7):
                    for l in range(7):
                        for m in range(2):
                            for n in range(4):
                                if self.Q[i,j,k,l,m,n] > 0:
                                    print(i,j,k,l,m,n)

    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=0.22):
        self.env = env
        self.states = self.get_states()
        self.actions = self.get_actions()
        self.n_a = len(self.actions)
        self.n_s = len(self.states)
        self.Q = np.zeros(( len(RANK), len(RANK), len(RANK), len(RANK), len([0,1]), len([0,1,2,3])  ))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def get_states(self):
        states = []
        for fi in RANK:
            for s in RANK:
                for t in RANK:
                    for fo in RANK:
                        state = [
                            RANK_VALUE[fi],
                            RANK_VALUE[s],
                            RANK_VALUE[t],
                            RANK_VALUE[fo]
                        ]
                        states.append(state)
        return states

    def get_actions(self):
        pick = list(range(0, 2))
        drop = list(range(0, 4))
        actions = []
        for p in pick:
            for d in drop:
                action = [p,d]
                actions.append(action)
        return actions

    def epsilon_greed(self, epsilon, s, type):
        i0 = s[0] - 1
        i1 = s[1] - 1
        i2 = s[2] - 1
        i3 = s[3] - 1
        if type == 'pick':
            if np.random.uniform() < epsilon:
                index = np.random.randint(2)
            else:
                index = np.where(self.Q[i0, i1, i2, i3, :, 0] == np.max(self.Q[i0, i1, i2, i3, :, 0]))[0][0]
        else:
            if np.random.uniform() < epsilon:
                index = np.random.randint(4)
            else:
                index = np.where(self.Q[i0, i1, i2, i3, 0, :] == np.max(self.Q[i0, i1, i2, i3, 0, :]))[0][0]
        return index


    def train(self, maxiter=10000):
        maxiter = maxiter
        w=0
        l=0
        debug = False
        for j in range(maxiter):
            # self.printQ()
            for player in rummy.players:
                player.points = player.stash_score()

            rummy.reset(rummy.players)
            random.shuffle(rummy.players)
            # int i = 0
            if debug:
                print(f'**********************************\n\t\tGame Starts : {j}\n***********************************')
            while not rummy.play():
                rummy._update_turn()
                if debug:
                    print(rummy.max_turns)
                for player in rummy.players:
                    if player.isBot:
                        if rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        rummy.computer_play(player)
                        if len(player.stash) == 0:
                            l+=1
                        if debug:
                            player.get_info(debug)
                            if len(player.stash) == 0:
                                print(f'{player.name} wins the round')

                    else:
                        if rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        player_info = player.get_info(debug)
                        #1s: pick ###################################################################################
                        # action_taken = np.random.choice(1)
                        epsilon=self.epsilon
                        alpha=self.alpha
                        gamma=self.gamma
                        i0_rank_to_val =player.stash[0].rank_to_val
                        i1_rank_to_val =player.stash[1].rank_to_val
                        i2_rank_to_val =player.stash[2].rank_to_val
                        card_pile_rank_to_val = rummy.pile[-1].rank_to_val
                        s = [
                            i0_rank_to_val,
                            i1_rank_to_val,
                            i2_rank_to_val,
                            card_pile_rank_to_val
                        ]
                        a = self.epsilon_greed(epsilon, s, type='pick')
                        if debug:
                            print(f'Card in pile {player_info["PileRank"]}{player_info["PileSuit"]}')
                        if debug:
                            print(f'{player.name} takes action {a}')
                        result_1 = rummy.pick_card(player, a)
                        r1 = result_1["reward"]
                        s1 = result_1["state"]
                        a1 = self.epsilon_greed(epsilon, s1, type='drop')
                        self.Q[s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, a, :] += alpha * (
                                r1 + gamma * self.Q[s1[0] - 1, s1[1] - 1, s1[2] - 1, s1[3] - 1, 0, a1] - self.Q[
                            s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, a, 0]
                        )
                        s = s1
                        a = a1
                        # self.printQ()
                        #1e: pick ###################################################################################
                        # player stash will have no cards if the player has melded them
                        # When you have picked up a card and you have drop it since the remaining cards have been melded.
                        if len(player.stash) == 1:
                            rummy.drop_card(player, player.stash[0])
                            w += 1
                            if debug:
                                print(f'{player.name} Wins the round')

                        elif len(player.stash) != 0:

                            player_info = player.get_info(debug)
                            if debug:
                                print(f'{player.name} takes action {a}')
                            # s = player_info['CardRanks']
                            #2s: drop ###################################################################################
                            # action_taken = np.random.choice(4)
                            # s = [player.stash[0].rank_to_val, player.stash[1].rank_to_val, player.stash[2].rank_to_val, player.stash[3].rank_to_val]
                            # a = self.epsilon_greed(0.1, s, type='drop')
                            card = player.stash[a]
                            if debug:
                                print(f'{player.name} drops card {card}')
                            result_1 = rummy.drop_card(player, card)
                            r1 = result_1["reward"]
                            card_pile_rank_to_val = rummy.pile[-1].rank_to_val
                            s1 = [
                                player.stash[0].rank_to_val,
                                player.stash[1].rank_to_val,
                                player.stash[2].rank_to_val,
                                card_pile_rank_to_val
                            ]
                            a1 = self.epsilon_greed(epsilon, s1, type='pick')
                            self.Q[s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, :, a] += alpha * (
                                    r1 + gamma * self.Q[s1[0] - 1, s1[1] - 1, s1[2] - 1, s1[3] - 1, a1, 0] -
                                    self.Q[s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, 0, a]
                            )
                            s=s1
                            a=a1
                            #2e: drop ###################################################################################
                        #                             pdb.set_trace()
                        else:
                            w += 1
                            if debug:
                                print(f'{player.name} Wins the round')
                        if debug:
                            player.get_info(debug)
            if rummy.max_turns <=0:
                score_bot = 0
                score_agent = 0
                for player in rummy.players:
                    if player.isBot:
                        score_bot = player.stash_score()
                    else:
                        score_agent = player.stash_score()
                if score_agent < score_bot:
                    w+=1
                elif score_agent > score_bot:
                    l+=1
                else:
                    w+=0.5
                    l+=0.5
        # if debug:
        print('====================================================', w, l)
        return self.Q


    def test(self):
        maxiter = 1
        w=0
        l=0
        debug = False
        for j in range(maxiter):
            for player in rummy.players:
                player.points = player.stash_score()
            rummy.reset(rummy.players)
            random.shuffle(rummy.players)
            if debug:
                print(f'**********************************\n\t\t Final Game Starts : \n***********************************')
            while not rummy.play():
                rummy._update_turn()
                if debug:
                    print(rummy.max_turns)
                for player in rummy.players:
                    if player.isBot:
                        if rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        rummy.computer_play(player)
                        if len(player.stash) == 0:
                            l+=1
                        if debug:
                            player.get_info(debug)
                            if len(player.stash) == 0:
                                print(f'{player.name} wins the round')

                    else:
                        if rummy.play():
                            continue
                        if debug:
                            print(f'{player.name} Plays')
                        player_info = player.get_info(debug)
                        #1s: pick ###################################################################################
                        # action_taken = np.random.choice(1)
                        epsilon=0
                        i0_rank_to_val =player.stash[0].rank_to_val
                        i1_rank_to_val =player.stash[1].rank_to_val
                        i2_rank_to_val =player.stash[2].rank_to_val
                        card_pile_rank_to_val = rummy.pile[-1].rank_to_val
                        s = [
                            i0_rank_to_val,
                            i1_rank_to_val,
                            i2_rank_to_val,
                            card_pile_rank_to_val
                        ]
                        a = self.epsilon_greed(epsilon, s, type='pick')
                        if debug:
                            print(f'Card in pile {player_info["PileRank"]}{player_info["PileSuit"]}')
                        if debug:
                            print(f'{player.name} takes action {a}')
                        result_1 = rummy.pick_card(player, a)
                        # r1 = result_1["reward"]
                        s1 = result_1["state"]
                        a1 = self.epsilon_greed(epsilon, s1, type='drop')
                        # self.Q[s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, a, :] += 0.1 * (
                        #         r1 + 0.99 * self.Q[s1[0] - 1, s1[1] - 1, s1[2] - 1, s1[3] - 1, 0, a1] - self.Q[
                        #     s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, a, 0]
                        # )
                        s = s1
                        a = a1
                        # self.printQ()
                        #1e: pick ###################################################################################

                        # player stash will have no cards if the player has melded them
                        # When you have picked up a card and you have drop it since the remaining cards have been melded.
                        if len(player.stash) == 1:
                            rummy.drop_card(player, player.stash[0])
                            w+=1
                            if debug:
                                print(f'{player.name} Wins the round')

                        elif len(player.stash) != 0:

                            player_info = player.get_info(debug)
                            if debug:
                                print(f'{player.name} takes action {a}')
                            # s = player_info['CardRanks']
                            #2s: drop ###################################################################################
                            # action_taken = np.random.choice(4)
                            # s = [player.stash[0].rank_to_val, player.stash[1].rank_to_val, player.stash[2].rank_to_val, player.stash[3].rank_to_val]
                            # a = self.epsilon_greed(0.1, s, type='drop')
                            card = player.stash[a]
                            if debug:
                                print(f'{player.name} drops card {card}')
                            result_1 = rummy.drop_card(player, card)
                            # r1 = result_1["reward"]
                            # card_pile_rank_to_val = rummy.pile[-1].rank_to_val
                            # s1 = [
                            #     player.stash[0].rank_to_val,
                            #     player.stash[1].rank_to_val,
                            #     player.stash[2].rank_to_val,
                            #     card_pile_rank_to_val
                            # ]
                            # a1 = self.epsilon_greed(0.05, s1, type='pick')
                            # self.Q[s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, :, a] += 0.1 * (
                            #         r1 + 0.99 * self.Q[s1[0] - 1, s1[1] - 1, s1[2] - 1, s1[3] - 1, a1, 0] -
                            #         self.Q[s[0] - 1, s[1] - 1, s[2] - 1, s[3] - 1, 0, a]
                            # )
                            # s=s1
                            # a=a1
                            #2e: drop ###################################################################################
                        #                             pdb.set_trace()
                        else:
                            w+=1
                            if debug:
                                print(f'{player.name} Wins the round')
                        if debug:
                            player.get_info(debug)
            if rummy.max_turns <=0:
                score_bot = 0
                score_agent = 0
                for player in rummy.players:
                    if player.isBot:
                        score_bot = player.stash_score()
                    else:
                        score_agent = player.stash_score()
                if score_agent < score_bot:
                    w+=1
                elif score_agent > score_bot:
                    l+=1
                else:
                    w+=0.5
                    l+=0.5
        return w, l




In [130]:
# Training with different alpha, gamma and epsilon
p1 = Player('jawad', list())
p2 = Player('comp1', list(), isBot=True)
rummy = RummyAgent([p1, p2], max_card_length=3, max_turns=20)

param_agent_list = []
alpha_gamma_epsilon_list =[]
option_alpha_list = [0.1, 0.2]
option_gamma_list = [0.9, 0.99]
option_epsilon_list = [0.1, 0.25]

for alpha in option_alpha_list:
    for gamma in option_gamma_list:
        for epsilon in option_epsilon_list:
            param = [alpha, gamma, epsilon]
            alpha_gamma_epsilon_list.append(param)

for param in alpha_gamma_epsilon_list:
    r = RLAgent(rummy, alpha=param[0], gamma=param[1], epsilon=param[2])
    q=r.train(maxiter=10000)
    param_agent_list.append([param, r])
    print('Done === ', str(param))

==================================================== 7915.5 2084.5
Done ===  [0.1, 0.9, 0.1]
==================================================== 7595.0 2405.0
Done ===  [0.1, 0.9, 0.25]
==================================================== 7905.0 2095.0
Done ===  [0.1, 0.99, 0.1]
==================================================== 7559.0 2441.0
Done ===  [0.1, 0.99, 0.25]
==================================================== 7858.0 2142.0
Done ===  [0.2, 0.9, 0.1]
==================================================== 7448.5 2551.5
Done ===  [0.2, 0.9, 0.25]
==================================================== 7776.5 2223.5
Done ===  [0.2, 0.99, 0.1]
==================================================== 7406.5 2593.5
Done ===  [0.2, 0.99, 0.25]


In [131]:
# Testing to find out the param(alpha, gamma, epsilon) based trained agent who performs best
param_score_agent_list = []
max_score = 0
max_param = []
max_agent = None
for elem in param_agent_list:
    param = elem[0]
    r = elem[1]
    
    s=0
    for n in range(1000):
        w, l = r.test()
        s+=w
    param_score_agent_list.append([param, s, r])
    if s > max_score:
        max_score = s
        max_param = param
        max_agent = r
    print('Done === ', str(param), " === ", str(s))

print('Max score: ', max_score)
print('Max param (alpha, gamma, epsilon): ', max_param)

Done ===  [0.1, 0.9, 0.1]  ===  820.5
Done ===  [0.1, 0.9, 0.25]  ===  818.5
Done ===  [0.1, 0.99, 0.1]  ===  831.5
Done ===  [0.1, 0.99, 0.25]  ===  844.5
Done ===  [0.2, 0.9, 0.1]  ===  824.0
Done ===  [0.2, 0.9, 0.25]  ===  812.0
Done ===  [0.2, 0.99, 0.1]  ===  841.5
Done ===  [0.2, 0.99, 0.25]  ===  807.0
Max score:  844.5
Max param (alpha, gamma, epsilon):  [0.1, 0.99, 0.25]


In [137]:
# Testing with only one agent (the best so far)
w=0
l=0
for i in range(1000):
    x, y = max_agent.test()
    w+=x
    l+=y
print('Max param: ', max_param)
print('win: ', w, ' vs lose: ', l)

Max param:  [0.1, 0.99, 0.25]
win:  817.5  vs lose:  182.5


# IV - Results

- Describe the choice of your parameters for $\gamma$, $\alpha$, and $\epsilon$. 
  - Run experiments to find good parameters
  - Show the experimental outputs to show the process of your selection
- Visualize the results and explain outputs (even from mine.)
  - Run the codes and tell me what you observe
  - Add more visualizations to enrich your explanation.

# V. Conclusions

<!-- Discuss the challenges or somethat that you learned. 
If you have any suggestion about the assignment, you can write about it.  -->

The major challenge that i faced in this assignment is how to define the states and the actions. 

The algorithm and other approaches are not that hard to implement as from the pseudocode,

but without defining the states and the actions properly its quite struggling to go ahead.

Another think i guess important to be careful about the states and actions while updating the Q values.

One thing i find interesting is to define different reward/penalty based on the different outcomes.

# Extra Credit for Tournament Top16!

- In two classes, we will have qualification and tournament.
- To participate the tournament, you need to use this [client](http://nbviewer.jupyter.org/url/webpages.uncc.edu/mlee173/teach/itcs6156/notebooks/assign/StudentClient.ipynb) codes.
- Locally you can run the [server](http://nbviewer.jupyter.org/url/webpages.uncc.edu/mlee173/teach/itcs6156/notebooks/assign/SERVER.ipynb) to verify your run. 

### Qualification Round 

- In qualification game, individual will play against a fixed policy agent on the server
  - You can access the server: TBA
  
  - Highly ranked (by the profit) will proceed to the tournament.
  - max. 32 players will be cut for main tournament (by 11 pm on Dec 1st).
  
### Tournament

- In tournament, 4 players will be play in each round.
  - In a table game, two best scoring players will proceed to next round
  - In final, the game will ends in 50 rounds and winner will be the one with lowest balance. 

## Grading

We will test your notebook with an additional grid.txt file. Please make sure to test in different maze input files. 

points | | description
--|--|:--
5 | Overview| states the objective and the appraoch 
35 | Methods | 
 |10| Review of the SARSA and Q-Learning
 | 5| Choice of TD learning and Reason
 | 5| Choice of Function Approximation and Reason
 |10| Implementation of the selected approach (RLAgent)
 | 5| explanation of the codes
45 | Results 
 |10| Reports the selected parameters 
 |15| Experimental outputs that show the choice of parameters. How do you choose them?
 |10| Output plots (5 for each)
 |10| reading and analysis of learning results and plots
10 | Participation to the tournament.
5 | Conclusions 
